In [1]:
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from obspy import read
import os
import sys
import numpy as np
import pandas as pd
from obspy.core import UTCDateTime
from shutil import copyfile
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score


In [2]:
def moving_average(l, N):
    sum = 0
    result = list( 0 for x in l)
 
    for i in range( 0, N ):
        sum = sum + l[i]
        result[i] = sum / (i+1)
 
    for i in range( N, len(l) ):
        sum = sum - l[i-N] + l[i]
        result[i] = sum / N
 
    return result
def msd_straight_forward(r):
    shifts = np.arange(len(r))
    msds = np.zeros(shifts.size)    

    for i, shift in enumerate(shifts):
        diffs = r[:-shift if shift else None] - r[shift:]
        sqdist = np.square(diffs).sum(axis=1)
        msds[i] = sqdist.mean()

    return msds

In [3]:
# data_folder = "E:\\obspy_data_processing\\OBSPY\\triggers\\new_triggers_upd"
data_folder = "E:\\obspy_data_processing\\OBSPY\\dataset"
# data_folder = "E:\\obspy_data_processing\\OBSPY\\copied"

In [15]:
stream

12 Trace(s) in Stream:
.6D97..HHZ | 2018-05-22T02:46:28.000000Z - 2018-05-22T02:47:13.000000Z | 100.0 Hz, 4501 samples
.6D98..HHZ | 2018-05-22T02:46:28.000000Z - 2018-05-22T02:47:13.000000Z | 100.0 Hz, 4501 samples
.6E02..HHZ | 2018-05-22T02:46:28.000000Z - 2018-05-22T02:47:13.000000Z | 100.0 Hz, 4501 samples
.6E09..HHZ | 2018-05-22T02:46:28.000000Z - 2018-05-22T02:47:13.000000Z | 100.0 Hz, 4501 samples
.6D97..HHN | 2018-05-22T02:46:28.000000Z - 2018-05-22T02:47:13.000000Z | 100.0 Hz, 4501 samples
.6D98..HHN | 2018-05-22T02:46:28.000000Z - 2018-05-22T02:47:13.000000Z | 100.0 Hz, 4501 samples
.6E02..HHN | 2018-05-22T02:46:28.000000Z - 2018-05-22T02:47:13.000000Z | 100.0 Hz, 4501 samples
.6E09..HHN | 2018-05-22T02:46:28.000000Z - 2018-05-22T02:47:13.000000Z | 100.0 Hz, 4501 samples
.6D97..HHE | 2018-05-22T02:46:28.000000Z - 2018-05-22T02:47:13.000000Z | 100.0 Hz, 4501 samples
.6D98..HHE | 2018-05-22T02:46:28.000000Z - 2018-05-22T02:47:13.000000Z | 100.0 Hz, 4501 samples
.6E02..HHE | 2018

In [4]:
dataset = list()
# for filename in os.listdir(data_folder)[:48]:
for filename in os.listdir(data_folder):
#     if filename.endswith("-cutted.msd"):
        print(filename)
        stream = read(data_folder+"\\" + filename)
#         label = filename[8:23]
        
#         date = label[0:4]+label[4:6]+label[6:8]+"T"+label[9:11]+":"+label[11:13]+":"+label[13:15]
#         print(date)
#         break
        label = filename[15:29]
        date = label[0:4]+label[4:6]+label[6:8]+"T"+label[8:10]+":"+label[10:12]+":"+label[12:14]
        try:
            peak_time = UTCDateTime(date)
        except:
            print ("Unexpected error:", sys.exc_info()[0])
            continue
        trace_times = [x.datetime.timestamp() for x in stream[0].times("utcdatetime")]
        trace_times_np = np.asarray(trace_times, dtype=np.int64)
        
        for trace in stream:
             dataset.append({'X':trace.data, 'Y': np.int64(peak_time.datetime.timestamp()-3600*7)})   
#         break

20180505140315-2018050514031862711035-augmented-cutted.msd
20180505140315-2018050514031862711035-cutted.msd
20180505140315-2018050514031862711530-augmented-cutted.msd
20180505140315-2018050514031862711530-cutted.msd
20180505140315-2018050514031862712025-augmented-cutted.msd
20180505140315-2018050514031862712025-cutted.msd
20180505140315-2018050514031862713015-augmented-cutted.msd
20180505140315-2018050514031862713015-cutted.msd
20180505140916-2018050514092007721035-augmented-cutted.msd
20180505140916-2018050514092007721035-cutted.msd
20180505140916-2018050514092007721530-augmented-cutted.msd
20180505140916-2018050514092007721530-cutted.msd
20180505140916-2018050514092007722025-augmented-cutted.msd
20180505140916-2018050514092007722025-cutted.msd
20180505140916-2018050514092007723015-augmented-cutted.msd
20180505140916-2018050514092007723015-cutted.msd
20180505230719-2018050523072205211035-augmented-cutted.msd
20180505230719-2018050523072205211035-cutted.msd
20180505230719-2018050523072

20180508180724-2018050818072958431530-augmented-cutted.msd
20180508180724-2018050818072958431530-cutted.msd
20180508180724-2018050818072958432025-augmented-cutted.msd
20180508180724-2018050818072958432025-cutted.msd
20180508180724-2018050818072958433015-augmented-cutted.msd
20180508180724-2018050818072958433015-cutted.msd
20180509062017-2018050906202113021035-augmented-cutted.msd
20180509062017-2018050906202113021035-cutted.msd
20180509062017-2018050906202113021530-augmented-cutted.msd
20180509062017-2018050906202113021530-cutted.msd
20180509062017-2018050906202113022025-augmented-cutted.msd
20180509062017-2018050906202113022025-cutted.msd
20180509062017-2018050906202113023015-augmented-cutted.msd
20180509062017-2018050906202113023015-cutted.msd
20180509130059-201805091301504771035-augmented-cutted.msd
20180509130059-201805091301504771035-cutted.msd
20180509130059-201805091301504771530-augmented-cutted.msd
20180509130059-201805091301504771530-cutted.msd
20180509130059-20180509130150477

20180511000440-2018051100044400981530-cutted.msd
20180511000440-2018051100044400982025-augmented-cutted.msd
20180511000440-2018051100044400982025-cutted.msd
20180511000440-2018051100044400983015-augmented-cutted.msd
20180511000440-2018051100044400983015-cutted.msd
20180511001139-2018051100114400431035-augmented-cutted.msd
20180511001139-2018051100114400431035-cutted.msd
20180511001139-2018051100114400431530-augmented-cutted.msd
20180511001139-2018051100114400431530-cutted.msd
20180511001139-2018051100114400432025-augmented-cutted.msd
20180511001139-2018051100114400432025-cutted.msd
20180511001139-2018051100114400433015-augmented-cutted.msd
20180511001139-2018051100114400433015-cutted.msd
20180511002306-201805110023968631035-augmented-cutted.msd
Unexpected error: <class 'ValueError'>
20180511002306-201805110023968631035-cutted.msd
Unexpected error: <class 'ValueError'>
20180511002306-201805110023968631530-augmented-cutted.msd
Unexpected error: <class 'ValueError'>
20180511002306-2018051

20180514144039-2018051414404345422025-cutted.msd
20180514144039-2018051414404345423015-augmented-cutted.msd
20180514144039-2018051414404345423015-cutted.msd
20180514144113-2018051414411708421035-augmented-cutted.msd
20180514144113-2018051414411708421035-cutted.msd
20180514144113-2018051414411708421530-augmented-cutted.msd
20180514144113-2018051414411708421530-cutted.msd
20180514144113-2018051414411708422025-augmented-cutted.msd
20180514144113-2018051414411708422025-cutted.msd
20180514144113-2018051414411708423015-augmented-cutted.msd
20180514144113-2018051414411708423015-cutted.msd
20180514144206-2018051414421070071035-augmented-cutted.msd
20180514144206-2018051414421070071035-cutted.msd
20180514144206-2018051414421070071530-augmented-cutted.msd
20180514144206-2018051414421070071530-cutted.msd
20180514144206-2018051414421070072025-augmented-cutted.msd
20180514144206-2018051414421070072025-cutted.msd
20180514144206-2018051414421070073015-augmented-cutted.msd
20180514144206-2018051414421

20180519081607-2018051908161045061530-augmented-cutted.msd
20180519081607-2018051908161045061530-cutted.msd
20180519081607-2018051908161045062025-augmented-cutted.msd
20180519081607-2018051908161045062025-cutted.msd
20180519081607-2018051908161045063015-augmented-cutted.msd
20180519081607-2018051908161045063015-cutted.msd
20180519082136-2018051908213961591035-augmented-cutted.msd
20180519082136-2018051908213961591035-cutted.msd
20180519082136-2018051908213961591530-augmented-cutted.msd
20180519082136-2018051908213961591530-cutted.msd
20180519082136-2018051908213961592025-augmented-cutted.msd
20180519082136-2018051908213961592025-cutted.msd
20180519082136-2018051908213961593015-augmented-cutted.msd
20180519082136-2018051908213961593015-cutted.msd
20180519234059-201805192341174961035-augmented-cutted.msd
20180519234059-201805192341174961035-cutted.msd
20180519234059-201805192341174961530-augmented-cutted.msd
20180519234059-201805192341174961530-cutted.msd
20180519234059-20180519234117496

20180522024655-2018052202465818611035-cutted.msd
20180522024655-2018052202465818611530-augmented-cutted.msd
20180522024655-2018052202465818611530-cutted.msd
20180522024655-2018052202465818612025-augmented-cutted.msd
20180522024655-2018052202465818612025-cutted.msd
20180522024655-2018052202465818613015-augmented-cutted.msd
20180522024655-2018052202465818613015-cutted.msd


In [5]:
df = pd.DataFrame(dataset)
X = df['X'].tolist()
Y = df['Y'].tolist()

In [7]:
seed = 7
test_size = 0.33

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state = seed)

In [16]:
clf = XGBRegressor(scale_pos_weight=4500)
clf.fit(X_train, y_train, verbose=True)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=4500, seed=None, silent=True,
       subsample=1)

In [24]:
y_pred = clf.predict(X_test)

In [28]:
y_pred = [np.int64(x) for x in y_pred]

In [25]:
mean_squared_error(y_test, y_pred)

162640899361.34604

In [29]:
y_pred

[1526011008,
 1526075776,
 1525946880,
 1525976448,
 1526041728,
 1526004224,
 1525974656,
 1526008832,
 1526369024,
 1526316800,
 1526056320,
 1526019456,
 1526049792,
 1526230272,
 1526533504,
 1526122496,
 1525982080,
 1526054528,
 1526204672,
 1526044672,
 1525891584,
 1526067456,
 1526534912,
 1526050688,
 1525957760,
 1526063488,
 1526003840,
 1526288128,
 1526401920,
 1526035840,
 1526021504,
 1526193920,
 1526070272,
 1525981312,
 1526036224,
 1526017152,
 1526103040,
 1526050432,
 1526225664,
 1526022016,
 1526483584,
 1525988352,
 1526349824,
 1526034560,
 1526284288,
 1526052992,
 1526053888,
 1526516864,
 1526012416,
 1526097792,
 1526044288,
 1526103936,
 1526034432,
 1526058112,
 1526006784,
 1526050944,
 1526073216,
 1526291968,
 1526009984,
 1526054528,
 1526052352,
 1526459776,
 1526064896,
 1525990912,
 1526269056,
 1526254208,
 1526201600,
 1526001920,
 1526014592,
 1526156288,
 1526157952,
 1526501376,
 1525971968,
 1526019072,
 1525992320,
 1525979904,
 1525987456,

In [26]:
y_test

[1526781104,
 1526258365,
 1525478960,
 1525511242,
 1525850659,
 1525550156,
 1525752449,
 1525928753,
 1526821668,
 1525947104,
 1526793512,
 1526093894,
 1525623164,
 1526662715,
 1526902433,
 1525945990,
 1526088178,
 1526667699,
 1525850051,
 1525649595,
 1525796421,
 1526326990,
 1526902344,
 1525511242,
 1526722877,
 1526667699,
 1526258443,
 1525947104,
 1526841169,
 1525541449,
 1525872358,
 1525871374,
 1525862784,
 1525478960,
 1526667370,
 1525945960,
 1525947104,
 1526792694,
 1525872358,
 1525623092,
 1525974042,
 1525622951,
 1525850051,
 1526093894,
 1526662715,
 1525898202,
 1526316494,
 1526902366,
 1526174215,
 1526662715,
 1526258365,
 1526258477,
 1526722877,
 1526277946,
 1525478598,
 1525820989,
 1525594014,
 1525871374,
 1526258365,
 1526093894,
 1526258443,
 1526821668,
 1525594014,
 1526174215,
 1526310314,
 1526902538,
 1525946684,
 1525680075,
 1525862784,
 1526245161,
 1526174215,
 1526902538,
 1526667370,
 1526173548,
 1525928591,
 1525796421,
 1525680075,

In [23]:
for x in np.sort(y_pred):
    print(UTCDateTime(x))

2018-05-08T00:25:36.000000Z
2018-05-08T04:13:52.000000Z
2018-05-08T09:18:56.000000Z
2018-05-08T12:01:04.000000Z
2018-05-08T12:01:04.000000Z
2018-05-08T12:43:44.000000Z
2018-05-08T16:25:36.000000Z
2018-05-08T16:25:36.000000Z
2018-05-08T18:42:08.000000Z
2018-05-08T20:43:44.000000Z
2018-05-08T22:21:52.000000Z
2018-05-09T00:14:56.000000Z
2018-05-09T01:38:08.000000Z
2018-05-09T04:01:04.000000Z
2018-05-09T05:07:12.000000Z
2018-05-09T06:19:44.000000Z
2018-05-09T08:42:40.000000Z
2018-05-09T08:42:40.000000Z
2018-05-09T10:10:08.000000Z
2018-05-09T13:28:32.000000Z
2018-05-09T13:52:00.000000Z
2018-05-09T14:11:12.000000Z
2018-05-09T15:38:40.000000Z
2018-05-09T16:59:44.000000Z
2018-05-09T17:31:44.000000Z
2018-05-09T17:33:52.000000Z
2018-05-09T17:53:04.000000Z
2018-05-09T18:08:00.000000Z
2018-05-09T18:16:32.000000Z
2018-05-09T18:29:20.000000Z
2018-05-09T18:46:24.000000Z
2018-05-09T19:56:48.000000Z
2018-05-09T20:48:00.000000Z
2018-05-09T22:04:48.000000Z
2018-05-09T22:06:56.000000Z
2018-05-09T23:10:56.

2018-05-11T15:25:52.000000Z
2018-05-11T15:25:52.000000Z
2018-05-11T15:25:52.000000Z
2018-05-11T15:25:52.000000Z
2018-05-11T15:25:52.000000Z
2018-05-11T15:28:00.000000Z
2018-05-11T15:28:00.000000Z
2018-05-11T15:30:08.000000Z
2018-05-11T15:30:08.000000Z
2018-05-11T15:30:08.000000Z
2018-05-11T15:32:16.000000Z
2018-05-11T15:32:16.000000Z
2018-05-11T15:32:16.000000Z
2018-05-11T15:36:32.000000Z
2018-05-11T15:38:40.000000Z
2018-05-11T15:40:48.000000Z
2018-05-11T15:40:48.000000Z
2018-05-11T15:40:48.000000Z
2018-05-11T15:42:56.000000Z
2018-05-11T15:42:56.000000Z
2018-05-11T15:42:56.000000Z
2018-05-11T15:42:56.000000Z
2018-05-11T15:42:56.000000Z
2018-05-11T15:42:56.000000Z
2018-05-11T15:45:04.000000Z
2018-05-11T15:47:12.000000Z
2018-05-11T15:47:12.000000Z
2018-05-11T15:47:12.000000Z
2018-05-11T15:49:20.000000Z
2018-05-11T15:49:20.000000Z
2018-05-11T15:51:28.000000Z
2018-05-11T15:51:28.000000Z
2018-05-11T15:51:28.000000Z
2018-05-11T15:53:36.000000Z
2018-05-11T15:53:36.000000Z
2018-05-11T15:53:36.

2018-05-14T00:19:12.000000Z
2018-05-14T00:32:00.000000Z
2018-05-14T00:34:08.000000Z
2018-05-14T00:36:16.000000Z
2018-05-14T00:38:24.000000Z
2018-05-14T00:38:24.000000Z
2018-05-14T00:49:04.000000Z
2018-05-14T00:51:12.000000Z
2018-05-14T00:57:36.000000Z
2018-05-14T01:04:00.000000Z
2018-05-14T01:08:16.000000Z
2018-05-14T01:08:16.000000Z
2018-05-14T01:18:56.000000Z
2018-05-14T01:21:04.000000Z
2018-05-14T01:21:04.000000Z
2018-05-14T01:44:32.000000Z
2018-05-14T02:10:08.000000Z
2018-05-14T02:27:12.000000Z
2018-05-14T02:33:36.000000Z
2018-05-14T03:20:32.000000Z
2018-05-14T03:20:32.000000Z
2018-05-14T03:26:56.000000Z
2018-05-14T03:37:36.000000Z
2018-05-14T03:37:36.000000Z
2018-05-14T03:41:52.000000Z
2018-05-14T03:48:16.000000Z
2018-05-14T03:50:24.000000Z
2018-05-14T03:54:40.000000Z
2018-05-14T04:30:56.000000Z
2018-05-14T04:33:04.000000Z
2018-05-14T04:35:12.000000Z
2018-05-14T04:52:16.000000Z
2018-05-14T04:56:32.000000Z
2018-05-14T05:11:28.000000Z
2018-05-14T05:26:24.000000Z
2018-05-14T05:32:48.